In [148]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import geopy
import networkx as nx
import random

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [149]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1] #this is a cycle
    assert set(tsp) == set(range(len(cities)))
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += dist_matrix[c1, c2]
    return tot_cost
        

In [150]:
cities_selection = 1 ##PARAMETER SELECTION
if cities_selection == 0:
    cities = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
elif cities_selection == 1:
    cities = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
elif cities_selection == 2:
    cities = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
elif cities_selection == 3:
    cities = pd.read_csv('cities/us.csv', header=None, names=['name', 'lat', 'lon'])
elif cities_selection == 4:
    cities = pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
    
    
dist_matrix = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geopy.distance.geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km


Initialization phase

In [151]:
def greedy_approach(starting_index):
    n = len(dist_matrix)
    unvisited = set(range(n))
    tour = [starting_index]  # Start from the first city
    unvisited.remove(starting_index)
        
    while unvisited:
        last = tour[-1]
        next_city = min(unvisited, key=lambda city: dist_matrix[last][city])
        tour.append(next_city)
        unvisited.remove(next_city)

    tour.append(starting_index)
    return tour
    

In [152]:
def random_init():
    tour = list(range(len(cities)))
    random.shuffle(tour)
    tour.append(tour[0])
    return tour
    

In [153]:
select_initialization = 2 ##PARAMETER SELECTION
if select_initialization == 1:
    tour = greedy_approach(10)
elif select_initialization == 2:
    tour = random_init()
else:
    print("Not valid initialization")
    

In [154]:
ordered_cities = cities.iloc[tour]
print(ordered_cities)
print(f"Found a path of {len(tour)-1} steps, total length {tsp_cost(tour):.2f}km")

                     name    lat    lon
39                  Terni  42.57  12.65
2                    Bari  41.12  16.87
17                Messina  38.19  15.55
5                 Bolzano  46.50  11.35
1                  Andria  41.23  16.29
42                  Turin  45.08   7.68
4                 Bologna  44.50  11.34
41                Trieste  45.65  13.77
44                 Verona  45.44  10.99
22                 Novara  45.45   8.62
20                  Monza  45.58   9.27
15                 Latina  41.47  12.89
19                 Modena  44.65  10.92
35                Salerno  40.68  14.77
10               Florence  43.78  11.24
36                Sassari  40.73   8.56
38                Taranto  40.48  17.24
12                  Forlì  44.22  12.03
25                  Parma  44.81  10.32
24                Palermo  38.12  13.36
13                  Genoa  44.42   8.93
16                Leghorn  43.55  10.30
11                 Foggia  41.47  15.55
3                 Bergamo  45.70   9.67


Evolutionary algorithm

In [155]:
def initialize_population(num_cities):
    max_population = min(num_cities * 2, 100) 
    population = []

    for i in range(max_population):
        start_index = i % num_cities  
        tour = greedy_approach(start_index)
        population.append(tour)

    return population

def tournament_selection(population, k=3):
    selected_indices = random.sample(list(range(len(population))), k)
    selected_tours = [population[i] for i in selected_indices]
    return min(selected_tours, key=tsp_cost)

def mutate(tour, num_swaps):
    n = len(tour)
    for _ in range(num_swaps):
        a, b = random.sample(range(1, n - 1), 2)
        tour[a], tour[b] = tour[b], tour[a]
    return tour

def local_search(path):
    best = path
    improved = True
    while improved:
        improved = False
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path) - 1):
                if j - i == 1: continue  # Skip adjacent cities
                new_path = best[:]
                new_path[i:j + 1] = reversed(best[i:j + 1])  # 2-opt swap
                if tsp_cost(new_path) < tsp_cost(best):
                    best = new_path
                    improved = True
    return best

def inv_crossover(parent1, parent2):
    child = parent1.copy()
    n = len(parent1) - 1 
    start = random.randint(1, n - 1)  
    
    start_value = parent1[start]
    index_in_parent2 = parent2.index(start_value)
    
    if start < index_in_parent2:
        segment_to_invert = parent1[start:index_in_parent2 + 1]
    else:
        segment_to_invert = parent1[index_in_parent2:start + 1]
    
    segment_to_invert.reverse()
    
    if start < index_in_parent2:
        child[start:index_in_parent2 + 1] = segment_to_invert
    else:
        child[index_in_parent2:start + 1] = segment_to_invert

    child[0] = child[-1] = child[0]
    
    return child


In [156]:
def mutation_algorithm(population, iterations=10000, initial_mutation_rate=0.6, num_swapping=6, cooling_rate=0.99):
    best_population = population
    best_fitness = tsp_cost(best_population)
    mutation_rate = initial_mutation_rate
    cooling_rate = 0.80

    for _ in range(iterations):
        new_population = best_population.copy()
        if random.random() < mutation_rate:
            new_population = mutate(new_population, num_swapping)
            new_population = local_search(new_population)
        
        new_fitness = tsp_cost(new_population)
    
        if new_fitness < best_fitness:
            best_population = new_population
            best_fitness = new_fitness
            # Decrease num_swapping if we're improving. In this way I improve exploitation if the solution is better
            num_swapping = max(1, num_swapping - 2)
        else:
            mutation_rate = min(1.0, mutation_rate*cooling_rate + abs(new_fitness - best_fitness) / 1000)
            # Increase num_swapping if not improving, encouraging more exploration.
            num_swapping = min(len(best_population) // 2, num_swapping + 1)
        
        # Decay the mutation rate over time. With a value equal to 0.80, the mutation rate is decreased of 20%
        mutation_rate *= cooling_rate

    return best_population


In [157]:
def crossover_algorithm(gen_number, path):
    population = []
    population.append(path)
    population.extend(initialize_population(len(cities)))
    
    for generation in range(gen_number):
        next_generation = []
        
        # Create new offspring
        while len(next_generation) < len(population):
            # Parent selection
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            
            # Perform crossover
            child = inv_crossover(parent1, parent2)
            
            next_generation.append(child)
        # Update population
        population = next_generation
        
        # Optionally print progress
        if generation % 10 == 0:
            best_fitness = min(tsp_cost(tour) for tour in population)
            print(f"Generation {generation}, Best Fitness: {best_fitness}")

    # Return the best solution found
    best_tour = min(population, key=tsp_cost)
    return best_tour

In [158]:
population = tour.copy()

algorithm_selection = 1 ##PARAMETER SELECTION
if algorithm_selection == 1:
    population = mutation_algorithm(population)
elif algorithm_selection == 2:
    population = crossover_algorithm(100, population)
else:
    print("Not valid algorithm")

ordered_cities = cities.iloc[population]
print(ordered_cities)
print(f"Found a path of {len(population)-1} steps, total length {tsp_cost(population):.2f}km")

                     name    lat    lon
39                  Terni  42.57  12.65
34                   Rome  41.89  12.50
15                 Latina  41.47  12.89
36                Sassari  40.73   8.56
7                Cagliari  39.22   9.10
24                Palermo  38.12  13.36
8                 Catania  37.50  15.08
37               Syracuse  37.07  15.29
17                Messina  38.19  15.55
31     Reggio di Calabria  38.11  15.65
21                 Naples  40.85  14.27
14  Giugliano in Campania  40.93  14.19
35                Salerno  40.68  14.77
38                Taranto  40.48  17.24
2                    Bari  41.12  16.87
1                  Andria  41.23  16.29
11                 Foggia  41.47  15.55
27                Pescara  42.46  14.21
0                  Ancona  43.60  13.50
41                Trieste  45.65  13.77
33                 Rimini  44.06  12.57
12                  Forlì  44.22  12.03
30                Ravenna  44.42  12.21
9                 Ferrara  44.84  11.61
